In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [7]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    "data/appendix-keywords.txt", embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

In [8]:
# RAG prompt
from langchain import hub

# Loads the latest version
prompt = hub.pull("rlm/rag-prompt", api_url="https://api.hub.langchain.com")

In [9]:
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()

retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke("Open Source 에 대해서 알려줘")

'Open Source는 소프트웨어, 하드웨어 또는 다른 유형의 제품 또는 서비스에서 내부적으로 사용하는 소스 코드를 공개하고 여러 사용자가 접근하고 수정할 수 있는 개방형 개발 방식을 의미합니다. 특히 소스 코드가 공개되어 있기 때문에 누구나 해당 소프트웨어를 자유롭게 사용, 복제, 수정, 배포할 수 있으며, 커뮤니티 형태로 개발된 소프트웨어 프로젝트가 주로 Open Source로 알려져 있습니다.'

In [5]:
from langchain.document_loaders import PyPDFLoader


# Step 1: 문서 로드
# loader = TextLoader("data/appendix-keywords.txt")
# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")
loader2 = PyPDFLoader("data/SPRi AI Brief_5월호_산업동향 최종.pdf")



In [10]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.document_loaders import PyPDFLoader


# Step 1: 문서 로드
# loader = TextLoader("data/appendix-keywords.txt")

# Step 1: 문서 로드
# loader = TextLoader("data/appendix-keywords.txt")
# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("data/SPRI_AI_Brief_2023년12월호_F.pdf")
loader2 = PyPDFLoader("data/SPRi AI Brief_5월호_산업동향 최종.pdf")

# Step 2: 문서 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # 500자로 문서를 분할
    chunk_overlap=50,  # 50자의 중복을 허용
    length_function=len,
)

docs1 = loader.load_and_split(text_splitter)
docs2 = loader.load_and_split(text_splitter)
docs = docs1 + docs2
# docs = loader.load_and_split(text_splitter)

# Step 3: 벡터 저장소 생성 & 임베딩(문장을 숫자 표현으로 바꾼다!!) -> 저장
vectorstore = FAISS.from_documents(docs, OpenAIEmbeddings())

# Step 4: 검색기(retriever) -> 나중에 질문(Query) 에 대한 유사도 검색을 하기 위함
retriever = vectorstore.as_retriever()

# Step 5: 프롬프트 작성, context: 검색기에서 가져온 문장, question: 질문
template = """당신은 문서에 대한 정보를 바탕으로 답변하는 친절한 Assistant 입니다. 무조건, 주어진 Context 바탕으로 답변해 주세요.
답변에 대한 출처도 함께 제공해 주세요.

#Context:
{context}

#Question: 
{question}
"""

# # Step 5: 프롬프트 작성, context: 검색기에서 가져온 문장, question: 질문
# template = """Answer the question based only on the following context:
# {context}

# Question: {question}
# """
prompt = ChatPromptTemplate.from_template(template)

# Step 6: OpenAI GPT-4 모델을 설정
model = ChatOpenAI(
    model="gpt-4o", streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
)

# Step 7: 질문에 대한 답변을 찾기 위한 체인 생성
retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# Step 8: 질문&답변
_ = retrieval_chain.invoke("삼성전자에서 자체 개발한 AI 의 이름은 뭐야?")

# # Step 8: 질문&답변
# _ = retrieval_chain.invoke("Word2Vec 에 대해서 알려줘")

삼성전자에서 자체 개발한 AI의 이름은 '삼성 가우스'입니다.

출처: SPRi AI Brief, 2023년 12월호, 페이지 12.